In [99]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [100]:
test = pd.read_csv(
    "/Users/jomaye/Documents/Programming/04-DS-repos/Python/spotify-api/datasets/country-uri-ds.csv", 
    sep=',' # Im CSV-File waren die Spalten via ","-Taste separiert, deshalb diese option zwingend anzugeben ist (ansonsten Error!)
) 
test # Unique ID == "country"-column...

,Country,Continent,URI
0,Global,Global,37i9dQZEVXbMDoHDwVN2tF
1,Argentina,South America,37i9dQZEVXbMMy2roB9myp
2,Australia,Australia,37i9dQZEVXbJPcfkRz0wJ0
3,Austria,Europe,37i9dQZEVXbKNHh6NIXu36
4,Belgium,Europe,37i9dQZEVXbJNSeeHswcKB
...,...,...,...
57,Turkey,Europe,37i9dQZEVXbIVYVBNw9D5K
58,United Kingdom,Europe,37i9dQZEVXbLnolsZ8PSNw
59,United States,North America,37i9dQZEVXbLRQDuF5jeBp
60,Uruguay,South America,37i9dQZEVXbMJJi3wgRbAy


In [101]:
unique_uri_names = test['URI'].unique()
unique_country_names = test['Country'].unique()
unique_continent_names = test['Continent'].unique()

In [104]:
unique_continent_names

array(['Global', 'South America', 'Australia', 'Europe', 'North America',
       'Asia'], dtype=object)

In [148]:
for (i, j, k) in zip(test['URI'][0:2], test["Country"][0:2], test["Continent"][0:2]): 
    print(i, j, k)

37i9dQZEVXbMDoHDwVN2tF Global Global
37i9dQZEVXbMMy2roB9myp Argentina South America


In [153]:
# Authentication in oder to use Spotify API
client_id = 'YOUR_PUBLIC_KEY_HERE' # Important here: I have hidden my credentials for accessing the Spotify-API, because you should NEVER publish such information on Github. Go and create your own public- & secret-API keys if you want to reproduce this project...
client_secret = 'YOUR_SECRET_KEY_HERE'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout=120) # <- THIS "requests_timeout = 120" is important, otherwise you will run into an error if you try to construct your (big) datasetvia the Spotify Web-API

In [157]:
# Create the Dataframe by using Spotify's Web-API (this code takes approximately 5 minutes!)

track_uri = []
track_name = []
artist_uri = []
#artist_info = []
artist_name = []
artist_pop = []
artist_genres = []
album = []
track_pop = []
duration = []
tempo = []
energy = []
danceability = []
playlist = []
continent = []
rank = []
for (i, j, k) in zip(test['URI'], test["Country"], test["Continent"]):
    playlist_URI = i # here, we dynamically place the URIs that we collected in our "test"-DF
    for el in range(1,51): # this creates a list from 1 to 50, which we will use to store all the values within the "rank"-array
        rank.append(el)
    for track in spotify.playlist_tracks(playlist_URI)["items"]:
        #URI of Tracks
        specific_track_uri = track["track"]["uri"] # here, we retrieve the "Track's ID" (for the "Track Features")
        track_uri.append(track["track"]["uri"])
        audio_features = spotify.audio_features(specific_track_uri)[0]

        #Track name
        track_name.append(track["track"]["name"])

        #Main Artist
        specific_artist_uri = track["track"]["artists"][0]["uri"]
        artist_uri.append(track["track"]["artists"][0]["uri"])

        artist_info = spotify.artist(specific_artist_uri) # here, we retrieve the "Artist's ID" (for the "general Popularity & Genre" of an artist)
    #    artist_info.append(sp.artist(artist_uri))

        #Name, popularity, genre
        artist_name.append(track["track"]["artists"][0]["name"])
        artist_pop.append(artist_info["popularity"]) # this can be retrieved by using the "artist's ID"
        artist_genres.append(artist_info["genres"]) # this can be retrieved by using the "artist's ID"

        # Track Features
        duration.append(audio_features["duration_ms"]) # this can be retrieved by using the "track's ID"
        tempo.append(audio_features["tempo"]) # this can be retrieved by using the "track's ID"
        energy.append(audio_features["energy"]) # this can be retrieved by using the "track's ID"
        danceability.append(audio_features["danceability"]) # this can be retrieved by using the "track's ID"

        #Album
        album.append(track["track"]["album"]["name"])

        #Popularity of the track
        track_pop.append(track["track"]["popularity"])
        
        # Meta-Infos
        playlist.append(j)
        continent.append(k)
    

In [163]:
# Create the DF by putting all the arrays together...

dict_for_df = dict() # create an empty dict
dict_for_df['Playlist'] = playlist
dict_for_df['Continent'] = continent
#dict_for_df['Rank'] = rank
dict_for_df['Artist Name'] = artist_name
dict_for_df['Album'] = album
dict_for_df['Track Name'] = track_name
dict_for_df['Artist Popularity'] = artist_pop
dict_for_df["Artist's Genres"] = artist_genres
dict_for_df['Track Popularity'] = track_pop
dict_for_df['Song Duration (ms)'] = duration
dict_for_df['Song Energy'] = energy
dict_for_df['Song Tempo'] = tempo
dict_for_df['Song Danceability'] = danceability
#dict_for_df['Track ID'] = track_uri
#dict_for_df['Artist ID'] = artist_uri

df = pd.DataFrame(dict_for_df) # now, we can create the DF
#df['Playlist'] = 'Global' # <-- THIS needs to change according to the name of the name of the countriy's Playlist
#df = df.reset_index() # this will be the song's ranking (for a specific week)
#df["index"] = df["index"] + 1
#df.rename({'index': 'Rank'}, axis='columns')
df

,Playlist,Continent,Artist Name,Album,Track Name,Artist Popularity,Artist's Genres,Track Popularity,Song Duration (ms),Song Energy,Song Tempo,Song Danceability
0,Global,Global,Miley Cyrus,Flowers,Flowers,91,"[dance pop, pop]",100,200455,0.681,117.999,0.707
1,Global,Global,Bizarrap,"Shakira: Bzrp Music Sessions, Vol. 53","Shakira: Bzrp Music Sessions, Vol. 53",90,"[argentine hip hop, pop venezolano, trap argen...",97,218289,0.632,122.104,0.778
2,Global,Global,SZA,SOS,Kill Bill,96,"[pop, r&b]",92,153947,0.735,88.980,0.644
3,Global,Global,Metro Boomin,HEROES & VILLAINS,Creepin' (with The Weeknd & 21 Savage),92,[rap],95,221520,0.620,97.950,0.715
4,Global,Global,Sam Smith,Gloria,Unholy (feat. Kim Petras),90,"[pop, uk pop]",60,156943,0.463,131.199,0.712
...,...,...,...,...,...,...,...,...,...,...,...,...
3084,Vietnam,Asia,NewJeans,NewJeans 1st EP 'New Jeans',Hype Boy,85,"[k-pop, k-pop girl group]",86,179027,0.935,99.976,0.593
3085,Vietnam,Asia,W/N,"3107 3 (feat. Nâu, titie, Dươngg)",3107 3,53,"[indie viet, v-pop]",57,240000,0.344,135.904,0.663
3086,Vietnam,Asia,JSOL,Sài Gòn Hôm Nay Mưa,Sài Gòn Hôm Nay Mưa,44,[v-pop],57,247059,0.604,136.077,0.624
3087,Vietnam,Asia,Low G,An Thần,An Thần,57,[vietnamese hip hop],56,174286,0.773,169.996,0.758


In [160]:
df[df.isnull().any(axis = 1)] # check, whether there is any missings?

,Playlist,Artist Name,Album,Track Name,Artist Popularity,Artist's Genres,Track Popularity,Song Duration (ms),Song Energy,Song Tempo,Song Danceability


In [173]:
df.to_csv(f"../datasets/scraped-spotify-dataset.csv", index=False) # use "index = False", otherwise - when you load the (saved) dataset - you will have a first row that is the index (twice), which you don't want!

In [174]:
testo = pd.read_csv(
    "/Users/jomaye/Documents/Programming/04-DS-repos/Python/spotify-api/datasets/scraped-spotify-dataset.csv", 
    sep=',' # Im CSV-File waren die Spalten via ","-Taste separiert, deshalb diese option zwingend anzugeben ist (ansonsten Error!)
) 
testo

,Playlist,Continent,Artist Name,Album,Track Name,Artist Popularity,Artist's Genres,Track Popularity,Song Duration (ms),Song Energy,Song Tempo,Song Danceability
0,Global,Global,Miley Cyrus,Flowers,Flowers,91,"['dance pop', 'pop']",100,200455,0.681,117.999,0.707
1,Global,Global,Bizarrap,"Shakira: Bzrp Music Sessions, Vol. 53","Shakira: Bzrp Music Sessions, Vol. 53",90,"['argentine hip hop', 'pop venezolano', 'trap ...",97,218289,0.632,122.104,0.778
2,Global,Global,SZA,SOS,Kill Bill,96,"['pop', 'r&b']",92,153947,0.735,88.980,0.644
3,Global,Global,Metro Boomin,HEROES & VILLAINS,Creepin' (with The Weeknd & 21 Savage),92,['rap'],95,221520,0.620,97.950,0.715
4,Global,Global,Sam Smith,Gloria,Unholy (feat. Kim Petras),90,"['pop', 'uk pop']",60,156943,0.463,131.199,0.712
...,...,...,...,...,...,...,...,...,...,...,...,...
3084,Vietnam,Asia,NewJeans,NewJeans 1st EP 'New Jeans',Hype Boy,85,"['k-pop', 'k-pop girl group']",86,179027,0.935,99.976,0.593
3085,Vietnam,Asia,W/N,"3107 3 (feat. Nâu, titie, Dươngg)",3107 3,53,"['indie viet', 'v-pop']",57,240000,0.344,135.904,0.663
3086,Vietnam,Asia,JSOL,Sài Gòn Hôm Nay Mưa,Sài Gòn Hôm Nay Mưa,44,['v-pop'],57,247059,0.604,136.077,0.624
3087,Vietnam,Asia,Low G,An Thần,An Thần,57,['vietnamese hip hop'],56,174286,0.773,169.996,0.758


In [175]:
testo[testo.isnull().any(axis = 1)] # check, whether there is any missings?

,Playlist,Continent,Artist Name,Album,Track Name,Artist Popularity,Artist's Genres,Track Popularity,Song Duration (ms),Song Energy,Song Tempo,Song Danceability


In [ ]:
# TODO: you will need to use Excel to make the "Rank"-Column, because it seems that the "Top 50" is not really a "Top 50" in each country (there are some missings)...